In [1]:
search_for = 133
start_from = 0
threads = 10
thread = 9
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.10522127151489258
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 25
all cases: 385308361
type: [1, 1, 1, 1, 133] 133
cases of this type: 312900721
100000 18.500675752976743
200000 29.455803913231577
300000 29.268024927072272
400000 28.914132047011773
500000 28.84802863903778
600000 30.116776099517857
700000 30.480111456411187
800000 28.823442438178365
900000 28.476584284649803
1000000 28.39336533234887
1100000 28.305347190352702
1200000 12.454429815648128
1300000 24.339123227076215
1400000 28.42353425643129
1500000 28.204767811312937
1600000 28.136282198831463
1700000 28.52567070798353
1800000 29.944720849961158
1900000 28.73846250133926
2000000 28.110083105734446
2100000 28.904550320803082
2200000 29.032613162333394
2300000 29.531482596584535
2400000 7.609454117317394
2500000 30.414593045223324
2600000 36.333249570998305
2700000 35.675581397682144
2800000 35.82619729706355
2900000 36.063598436655674
300000

29400000 19.04488113733638
29500000 16.596265884113727
29600000 26.212784652021718
29700000 27.434045112499263
29800000 27.725544528655348
29900000 28.84083699211908
30000000 27.491982568708476
30100000 28.420843494305384
30200000 29.226072163724115
30300000 28.751318100194403
30400000 12.678427143478022
30500000 28.98967893644982
30600000 29.232612761465923
30700000 30.553823925111576
30800000 26.203362276402494
30900000 15.0408459778661
31000000 29.078270321362215
31100000 29.616505199592478
31200000 28.80685354533639
31300000 28.500082761221524
31400000 28.27947275797722
31500000 28.189970865975045
31600000 27.755232829484207
31700000 26.77775603273221
31800000 7.955228494634903
31900000 27.678719747952364
32000000 26.833721122179302
32100000 28.233292674388025
32200000 28.39549129507602
32300000 28.9340353038189
32400000 28.55159602016157
32500000 29.761541130222074
32600000 29.057429452004662
32700000 19.87404723631716
32800000 21.903052976070644
32900000 29.454681696471173
330000

59200000 27.290165697354567
59300000 11.488153724689148
59400000 24.65662988443315
59500000 25.205806003056885
59600000 25.510161735068802
59700000 23.81895124952039
59800000 23.757049472720567
59900000 24.014512350615245
60000000 13.83015912263504
60100000 17.874295750449434
60200000 24.587540741818827
60300000 23.848772172326473
60400000 23.87582407404778
60500000 25.280172050609067
60600000 24.987243310567724
60700000 21.478623784175348
60800000 16.47970422415182
60900000 27.023602690881855
61000000 27.003264169438705
61100000 26.59308864632901
61200000 27.155278038734185
61300000 27.029078901580117
61400000 26.29595814433707
61500000 26.822079950529396
61600000 25.570838387430133
61700000 10.173428521846475
61800000 25.440448128961762
61900000 25.13115152361541
62000000 24.304646480639594
62100000 23.801432353370743
62200000 23.51069699805034
62300000 24.266865072197476
62400000 6.074431074715775
62500000 24.61097907204127
62600000 23.397871584767383
62700000 23.427555881663068
628

88900000 22.21340242432829
89000000 22.896323162205462
89100000 23.23763075108775
89200000 23.988127174108758
89300000 23.007998878269326
89400000 24.193164384096697
89500000 23.887644111747587
89600000 23.627363356841748
89700000 23.765285545577598
89800000 23.35472870340973
89900000 22.40454434260448
90000000 21.849840388385342
90100000 12.046281401961728
90200000 17.536234773030927
90300000 21.058495501251166
90400000 26.31195794272039
90500000 22.602656252047858
90600000 9.330572052406986
90700000 18.97654606217245
90800000 21.666219773987006
90900000 20.70771046396815
91000000 21.089708951262356
91100000 10.64924300756182
91200000 20.028569827018607
91300000 22.841910678265858
91400000 21.871104342403864
91500000 23.28533375007808
91600000 22.66292153788452
91700000 24.18226598537057
91800000 23.784845822424906
91900000 23.569564270289288
92000000 23.38779694433415
92100000 24.304288806514997
92200000 22.93122266694732
92300000 22.503252090367237
92400000 21.99052037803478
9250000

118000000 20.417050103600918
118100000 19.533152030225114
118200000 19.710568246596917
118300000 18.89558704296814
118400000 19.041182010855813
118500000 16.736274210059886
118600000 9.46486175576788
118700000 19.318249657776043
118800000 12.92111468231041
118900000 11.453490304082884
119000000 18.52796782873017
119100000 14.540760931734038
119200000 15.478701405103536
119300000 19.098257363042173
119400000 18.56406006374059
119500000 19.424109461563642
119600000 19.46920697445735
119700000 19.68126547018551
119800000 20.386622231086186
119900000 21.491294751065173
120000000 20.623263352397107
120100000 21.628683829365933
120200000 21.260155314809456
120300000 20.77395230975739
120400000 19.531982898570526
120500000 19.47251375166419
120600000 19.665131914434436
120700000 19.011849403985167
120800000 19.185934648667704
120900000 12.343854670758953
121000000 13.65280106886271
121100000 18.3278420245675
121200000 5.461096166344082
121300000 19.159359191909516
121400000 16.953038604076987

...

261300000 10.506983026378094
261400000 10.158986664370213
261500000 8.394432113544251
261600000 6.394783191745522
261700000 9.825022342780878
261800000 10.11870018402145
261900000 9.415280014711131
262000000 9.997307534371929
262100000 8.990688506033969
262200000 9.700055545551713
262300000 7.213191620340193
262400000 4.785932067862248
262500000 9.609111338779448
262600000 9.058355779079577
262700000 9.888864887271264
262800000 9.376228573065712
262900000 9.493944278937793
263000000 10.281730729460504
263100000 7.990926641340413
263200000 6.314901795799782
263300000 9.935654185482194
263400000 10.451928199436015
263500000 10.060000128299285
263600000 10.67581443582383
263700000 9.72813723232269
263800000 10.491118182715196
263900000 3.810427688587036
264000000 10.424751930550238
264100000 10.287782259728678
264200000 9.145855125519466
264300000 9.864770577441657
264400000 9.209560162179043
264500000 9.443280289625063
264600000 8.740216783998246
264700000 3.4346596516507626
264800000 8.

290600000 1.7552263555500731
290700000 7.613374694582529
290800000 7.71648613908571
290900000 7.768405510717343
291000000 7.653452582641062
291100000 7.669545537720276
291200000 7.8067531785963125
291300000 7.778147836966293
291400000 7.831572998134987
291500000 7.882780227288562
291600000 2.847835662591845
291700000 8.030525804127652
291800000 7.749972675603233
291900000 4.201774961752521
292000000 6.991315089383797
292100000 7.731070754515615
292200000 7.479440053162293
292300000 7.6343088422302765
292400000 8.94561928017515
292500000 9.278525647611614
292600000 7.582177805360592
292700000 7.398966682450508
292800000 7.253021192476354
292900000 4.799827301619909
293000000 4.643734740705481
293100000 7.217249318684203
293200000 7.421277862223403
293300000 7.37011862569626
293400000 7.46904968774663
293500000 7.675023071379429
293600000 7.620076081070587
293700000 7.3948733530576645
293800000 7.598186355087479
293900000 7.059338094323671
294000000 3.5876128703406978
294100000 7.4064242

319500000 2.3201012459206436
319600000 2.294391644630563
319700000 2.3093538982124
319800000 2.4386342531865695
319900000 2.510294207274598
320000000 2.729185278082824
320100000 2.4574440807174023
320200000 2.636484015835927
320300000 2.268992321708052
320400000 2.2863735442318536
320500000 2.482297935716438
320600000 2.3224765679348756
320700000 2.052608154891111
320800000 2.3318193379642804
320900000 2.1110532668537405
321000000 1.6826308593826613
321100000 1.2756890597371877
321200000 1.2782402688406684
321300000 1.2572231507119211
321400000 1.4514868778211116
321500000 2.4025385108851554
321600000 2.676100072732704
321700000 2.161514727301729
321800000 1.9729691523275419
321900000 2.3262971803868746
322000000 2.3489037008442244
322100000 2.249776263970677
322200000 2.26070850637819
322300000 2.6323699948025467
322400000 2.4128434463524746
322500000 2.5213871915628983
322600000 2.4215448816515464
322700000 2.348397348917474
322800000 2.12824996202596
322900000 2.1312739072978895
323

348100000 1.420024240309137
348200000 1.4722126101163076
348300000 1.580029170660009
348400000 1.3925922698230087
348500000 1.3784477298988007
348600000 1.3403783839451182
348700000 1.377550454362474
348800000 1.1535177992378784
348900000 1.2254072905533235
349000000 1.4442831015457234
349100000 1.3922462280199295
349200000 0.9912190096406494
349300000 0.6865204102649867
349400000 0.7102572516289115
349500000 0.7437286573490473
349600000 0.9066121270404637
349700000 1.082094640494109
349800000 1.4367285501779694
349900000 1.1955573096715788
350000000 1.2887742913563331
350100000 1.3369040162248116
350200000 1.246500876146549
350300000 1.2767958224434455
350400000 1.3982166508810467
350500000 1.339880042831687
350600000 1.4557241837712758
350700000 1.323129003134362
350800000 1.3369084277626853
350900000 1.2450289546896556
351000000 1.2487558664346443
351100000 1.2384924259103536
351200000 1.2535781140019397
351300000 1.2406123610974908
351400000 1.2721897403525173
351500000 1.002144250

375900000 0.05948517658826113
376000000 0.05535931759325266
376100000 0.05808762145210041
376200000 0.05791578779902658
376300000 0.04900882507780194
376400000 0.05489938279939757
best so far: 0
type: [1, 1, 7, 1, 19] 133
cases of this type: 6385729
376500000 0.2526039539413909
376600000 0.3302827377269129
376700000 0.2866003992777261
376800000 0.3177353153357433
376900000 0.2861990525919119
377000000 0.3086368551068127
377100000 0.30566990932680366
377200000 0.24154356977698208
377300000 0.21212392402042943
377400000 0.18701127303200693
377500000 0.18898765760789515
377600000 0.2009708064504544
377700000 0.20359470650787354
377800000 0.20065883544790347
377900000 0.20198793977846238
378000000 0.2101493884512802
378100000 0.17491731889657974
378200000 0.17424080544549952
378300000 0.20181305078157188
378400000 0.18774455134123363
378500000 0.22141470926127568
378600000 0.19177302375092375
378700000 0.23280806869663992
378800000 0.1898975186579936
378900000 0.046988355215575955
37900000